# Add Feature Recommender on H&M Data

In this notebook we train an SNQN model to recomend a ranked list of items to users in the dataset. We are using H&M's e-commerce data to train this model.

1. Clone the git repository containing all the source code

In [1]:
# cloning the git repository
!git clone https://github.com/szheng3/recommendation-system.git

fatal: destination path 'recommendation-system' already exists and is not an empty directory.


2. Install the missing libraries (specifically for google colab)

In [2]:
# installing missing libraries (specifically for google colab)
!pip install trfl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


3. Download the H&M Dataset transaction log file from AWS S3 bucket

In [3]:
# downloading the "transactions_train.csv" datafile from the H&M Dataset
!wget https://aipi590.s3.amazonaws.com/transactions_train.csv -P "/content/recommendation-system/ItemFeatures/HM/data/"

--2023-05-01 20:33:27--  https://aipi590.s3.amazonaws.com/transactions_train.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 54.231.162.121, 52.216.162.139, 3.5.1.162, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|54.231.162.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488002253 (3.2G) [text/csv]
Saving to: ‘/content/recommendation-system/ItemFeatures/HM/data/transactions_train.csv’

transactions_train. 100%[===================>]   3.25G  58.5MB/s    in 54s     

2023-05-01 20:34:21 (61.4 MB/s) - ‘/content/recommendation-system/ItemFeatures/HM/data/transactions_train.csv’ saved [3488002253/3488002253]



4. Run the "gen_replay_buffer.py" script to pre-process data and generate replay buffer

In [4]:
# Executing the python script "gen_replay_buffer" to sample the given H&M dataset
!python "/content/recommendation-system/ItemFeatures/HM/src/gen_replay_buffer.py" --size=20000 --format=paper --data="/content/recommendation-system/ItemFeatures/HM/data/"


Start reading all transaction data ...
Finish reading in 00:00:35

Filter and save all valid sampled data
/content/recommendation-system/ItemFeatures/HM/src/gen_replay_buffer.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_sessions['valid_session'] = sampled_sessions.session_id.map(sampled_sessions.groupby('session_id')['item_id'].size() > 2)
Index(['timestamp', 'session_id', 'item_id', 'price', 'sales_channel_id'], dtype='object')

Start counting popularity ...
211141it [00:09, 21957.58it/s]
Popularity finished in 00:00:09

Start spliting into train, val, test data ...

           Generate Replay Buffer:
                Total Session Size : 20000
                     Train:      14000 ids | 148615 actions
                     Vali

5. Run script to begin training and evaluate the SASRec-SNQN model.

In [ ]:
# Executing the python script "SNQN_v2.py" to train a SNQN model on the sampled data and evaluate it on validation data using HR (hit rate) and NDCG metrics for different values of k
!ls /content/recommendation-system/ItemFeatures/HM/data/
!python "/content/recommendation-system/ItemFeatures/HM/src/SNQN_new.py" --model=GRU --data="/content/recommendation-system/ItemFeatures/HM/data/" --epoch=2

data_statis.df	replay_buffer.df      sampled_test.df	transactions_train.csv
pop_dict.txt	sampled_sessions.csv  sampled_train.df
readme.md	sampled_sessions.df   sampled_val.df
2023-05-01 20:36:14.524199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 20:36:15.791596: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/recommendation-system/ItemFeatures/HM/src/SNQN_new.py:134: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  tf.compat.v1.nn.rnn_cell.GRUCell(self.hidden_size),
2023-05-01 20:36:18.554927: I tensorflow/compiler/xla/stream_executor/c